# Setup

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!pip install spacy -q

In [4]:
!python -m spacy download fr_core_news_sm -q
!python -m spacy download en_core_web_sm -q
!python -m spacy download de_core_news_sm -q

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-02-26 13:07:02.030748: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-26 13:07:04.186821: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-26 13:07:04.186974: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open share

In [5]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import pickle

# import matplotlib.pyplot as plt
# %matplotlib inline
# import matplotlib.ticker as ticker
import numpy as np

import spacy
from tqdm import tqdm

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = "cuda" if torch.cuda.is_available() else "cpu"
device

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


'cpu'

In [6]:
spacy_fr = spacy.load('fr_core_news_sm')
spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

In [7]:
def load_data(filename: str):
    with open(filename, 'rb') as f:
        dataset = pickle.load(f)
        print('load_data SUCCESS')
        return dataset

# Align with 3 langs at same time

In [10]:
enfr_set = load_data('/content/gdrive/MyDrive/Colab Notebooks/eaai24/Datasets/en-fr.pkl')

load_data SUCCESS


In [11]:
deen_set = load_data('/content/gdrive/MyDrive/Colab Notebooks/eaai24/Datasets/de-en.pkl')
defr_set = load_data('/content/gdrive/MyDrive/Colab Notebooks/eaai24/Datasets/de-fr.pkl')

load_data SUCCESS
load_data SUCCESS


In [12]:
len(enfr_set), len(deen_set), len(defr_set)

(2051014, 1961119, 1942666)

In [25]:
i = 500
# print(enfr_set[i])
print(deen_set[459])
print(defr_set[444])

{'de': 'Was uns hier im Parlament betrifft, so möchte ich auf den im Mai verabschiedeten und für Parlament und Kommission geltenden Verhaltenskodex verweisen.', 'en': 'As for us in Parliament, I would like to remind you of the code of conduct between the Commission and Parliament which was signed in May.'}
{'de': 'Das war für uns neue Mitglieder das erste Mal, und ich sehe darin einen äußerst interessanten Prozeß.', 'fr': "C' était pour nous, nouveaux députés, la première fois, et c' est un processus extrêmement intéressant."}


In [26]:
deen_pairs = [[pair['en'], pair['de']] for pair in deen_set]
defr_pairs = [[pair['de'], pair['fr']] for pair in defr_set]

In [42]:
de2_dict = {pair[0]:i for i, pair in enumerate(defr_pairs)}

In [43]:
align_idx_list = []
for i, pair1 in tqdm(enumerate(deen_pairs)):
  en_sent, de_sent1 = pair1
  de_sent2_idx = de2_dict.get(de_sent1, -1)
  align_idx_list.append((i, de_sent2_idx))

1961119it [00:04, 424758.77it/s]


In [45]:
align_idx_list_unk_rmved = [(k, v) for k, v in align_idx_list if v!=-1]
len(align_idx_list_unk_rmved), align_idx_list_unk_rmved[:10]

(1818194,
 [(0, 1933280),
  (1, 1),
  (3, 4),
  (4, 5),
  (5, 204436),
  (6, 1930472),
  (7, 1892568),
  (8, 9),
  (9, 10),
  (10, 11)])

In [46]:
save_endefr = []
thres = 0.5
for k, v in tqdm(align_idx_list_unk_rmved):
  en_sent, de_sent1 = deen_pairs[k]
  de_sent2, fr_sent = defr_pairs[v]
  save_endefr.append({
      'en': en_sent,
      'de': de_sent2 if random.random() < thres else de_sent1,
      'fr': fr_sent
  })

100%|██████████| 1818194/1818194 [00:04<00:00, 430380.33it/s]


In [51]:
len(save_endefr), save_endefr[-198]

(1818194,
 {'en': 'The Commission cannot, however, accept Amendment No 4, even in its new, modified version.',
  'de': 'Allerdings kann sie Änderungsantrag 4, auch in seiner neuen, verbesserten Fassung, nicht annehmen.',
  'fr': "En revanche, la Commission ne peut accepter l' amendement 4, même dans sa nouvelle version modifiée."})

In [50]:
with open('/content/gdrive/MyDrive/Colab Notebooks/eaai24/Datasets/en-de-fr.pkl', 'wb') as f:
  pickle.dump(save_endefr, f)

# Load data

In [56]:
PAD_token = 0
SOS_token = 1
EOS_token = 2
# MIN_LENGTH = 2
# MAX_LENGTH = 50

class Lang:
    def __init__(self, name):
        self.name = name
        # self.tkz = spacy.load('fr_core_news_sm') if name=='fr' else spacy.load('en_core_web_sm')
        self.max_len = 0
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0:"<PAD>", 1: "<SOS>", 2: "<EOS>"}
        self.n_words = 3  # Count SOS and EOS

    # def addSentence(self, sentence):
    #   words = [tok.text for tok in self.tkz.tokenizer(sentence)]
    #   # for word in sentence.split(' '):
    #   for word in words:
    #       self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index: # if not in dict:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else: # count++ if word already in dict
            self.word2count[word] += 1

In [57]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = s.lower().strip()
    # s = unicodeToAscii(s)
    # s = re.sub(r"([.!?])", r" \1", s)
    # s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def readLangs(lang1='en', lang2='fr', reverse=False):
    print("Reading lines...")

    dataset = load_data('/content/gdrive/MyDrive/Colab Notebooks/eaai24/Datasets/en-fr.pkl')
    # dataset = open('/content/eng-fra.txt', encoding='utf-8').read().strip().split('\n')

    pairs = [[normalizeString(pair[lang1]), normalizeString(pair[lang2])] for pair in dataset]
    # pairs = [[normalizeString(s) for s in l.split('\t')] for l in dataset]
    input_lang = Lang(lang1)
    output_lang = Lang(lang2)

    return (input_lang, output_lang, pairs)

def read3Langs(lang1='en', lang2='de', lang3='fr'):
    print("Reading lines...")

    dataset = load_data('/content/gdrive/MyDrive/Colab Notebooks/eaai24/Datasets/en-de-fr.pkl')

    pairs = [[normalizeString(pair[lang1]), normalizeString(pair[lang2]), normalizeString(pair[lang3])] for pair in dataset]
    # pairs = [[normalizeString(s) for s in l.split('\t')] for l in dataset]
    input_lang = Lang(lang1)
    pivot_lang = Lang(lang2)
    output_lang = Lang(lang3)

    return (input_lang, pivot_lang, output_lang, pairs)

In [ ]:
# def prepareData(lang1='en', lang2='fr'):
#   (input_lang, output_lang, pairs) = readLangs(lang1, lang2)
#   print("Read %s sentence pairs" % len(pairs))
#   # pairs = filterPairs(pairs)
#   # print("Trimmed to %s sentence pairs" % len(pairs))

#   print("Counting words...")
#   for i in range(len(pairs)):
#     in_sent, out_sent = pairs[i]

#     en_words = [tok.text for tok in spacy_en.tokenizer(in_sent)]
#     input_lang.max_len = max(input_lang.max_len, len(en_words))
#     for en_word in en_words:
#       input_lang.addWord(en_word)

#     fr_words = [tok.text for tok in spacy_fr.tokenizer(out_sent)]
#     output_lang.max_len = max(output_lang.max_len, len(fr_words))
#     for fr_word in fr_words:
#       output_lang.addWord(fr_word)
    
#     pairs[i] = ['#'.join(en_words), '#'.join(fr_words)]
  
#   print("Counted words:")
#   print(input_lang.name, input_lang.n_words)
#   print(output_lang.name, output_lang.n_words)

#   return (input_lang, output_lang, pairs)

## For 2 langs

In [ ]:
input_lang, output_lang, pairs = readLangs()

Reading lines...
load_data SUCCESS


In [ ]:
input_lang.name, output_lang.name, pairs[2]

('en',
 'fr',
 ["although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.",
  'comme vous avez pu le constater, le grand "bogue de l\'an 2000" ne s\'est pas produit. en revanche, les citoyens d\'un certain nombre de nos pays ont été victimes de catastrophes naturelles qui ont vraiment été terribles.'])

In [ ]:
len(pairs)

2051014

In [ ]:
for i in tqdm(range(len(pairs))):
    in_sent, out_sent = pairs[i]

    en_words = [tok.text for tok in spacy_en.tokenizer(in_sent)]
    input_lang.max_len = max(input_lang.max_len, len(en_words))
    for en_word in en_words:
      input_lang.addWord(en_word)

    fr_words = [tok.text for tok in spacy_fr.tokenizer(out_sent)]
    output_lang.max_len = max(output_lang.max_len, len(fr_words))
    for fr_word in fr_words:
      output_lang.addWord(fr_word)
    
    pairs[i] = ['#'.join(en_words), '#'.join(fr_words)]

100%|██████████| 2051014/2051014 [07:55<00:00, 4310.56it/s]


In [ ]:
pairs[2]

["although#,#as#you#will#have#seen#,#the#dreaded#'#millennium#bug#'#failed#to#materialise#,#still#the#people#in#a#number#of#countries#suffered#a#series#of#natural#disasters#that#truly#were#dreadful#.",
 'comme#vous#avez#pu#le#constater#,#le#grand#"#bogue#de#l\'#an#2000#"#ne#s\'#est#pas#produit#.#en#revanche#,#les#citoyens#d\'#un#certain#nombre#de#nos#pays#ont#été#victimes#de#catastrophes#naturelles#qui#ont#vraiment#été#terribles#.']

## For 3 langs

In [58]:
(input_lang, pivot_lang, output_lang, pairs) = read3Langs(lang1='en', lang2='de', lang3='fr')

Reading lines...
load_data SUCCESS


In [59]:
input_lang.name, pivot_lang.name, output_lang.name, pairs[3212]

('en',
 'de',
 'fr',
 ['in a symbolic but significant gesture commissioners have voluntarily renounced their entitlement to the tax-free purchase of alcohol, tobacco, petrol and consumer goods.',
  'in einer symbolischen, jedoch bedeutenden geste haben die mitglieder der kommission ihren freiwilligen verzicht auf ihren anspruch auf den steuerfreien einkauf von alkohol, tabak, benzin und konsumgütern bekanntgegeben.',
  "de manière symbolique mais néanmoins significative, les commissaires ont volontairement renoncé à bénéficier des achats hors taxes d'alcool, de tabac, d'essence et de produits de consommation."])

In [60]:
len(pairs)

1818194

In [61]:
for i in tqdm(range(len(pairs))):
    in_sent, piv_sent, out_sent = pairs[i]

    en_words = [tok.text for tok in spacy_en.tokenizer(in_sent)]
    input_lang.max_len = max(input_lang.max_len, len(en_words))
    for en_word in en_words:
      input_lang.addWord(en_word)
    
    de_words = [tok.text for tok in spacy_de.tokenizer(piv_sent)]
    pivot_lang.max_len = max(pivot_lang.max_len, len(de_words))
    for de_word in de_words:
      pivot_lang.addWord(de_word)

    fr_words = [tok.text for tok in spacy_fr.tokenizer(out_sent)]
    output_lang.max_len = max(output_lang.max_len, len(fr_words))
    for fr_word in fr_words:
      output_lang.addWord(fr_word)
    
    pairs[i] = ['#'.join(en_words), '#'.join(de_words), '#'.join(fr_words)]

100%|██████████| 1818194/1818194 [13:09<00:00, 2301.93it/s]


In [62]:
pairs[3212]

['in#a#symbolic#but#significant#gesture#commissioners#have#voluntarily#renounced#their#entitlement#to#the#tax#-#free#purchase#of#alcohol#,#tobacco#,#petrol#and#consumer#goods#.',
 'in#einer#symbolischen#,#jedoch#bedeutenden#geste#haben#die#mitglieder#der#kommission#ihren#freiwilligen#verzicht#auf#ihren#anspruch#auf#den#steuerfreien#einkauf#von#alkohol#,#tabak#,#benzin#und#konsumgütern#bekanntgegeben#.',
 "de#manière#symbolique#mais#néanmoins#significative#,#les#commissaires#ont#volontairement#renoncé#à#bénéficier#des#achats#hors#taxes#d'#alcool#,#de#tabac#,#d'#essence#et#de#produits#de#consommation#."]

# Sort freq dict

## For 2 langs

In [ ]:
len(input_lang.word2count), len(output_lang.word2count)

(103940, 140537)

In [ ]:
sorted_freq_en = {k: v for k, v in sorted(input_lang.word2count.items(), key=lambda item: item[1], reverse=True)}
sorted_freq_fr = {k: v for k, v in sorted(output_lang.word2count.items(), key=lambda item: item[1], reverse=True)}
len(sorted_freq_en), len(sorted_freq_fr)  # (103940, 140537)

In [ ]:
for i, (k, v) in enumerate(sorted_freq_en.items()):
  print(k, v)
  if i==5: break
print()
for i, (k, v) in enumerate(sorted_freq_fr.items()):
  print(k, v)
  if i==5: break

the 3942610
, 2733859
. 2014253
of 1868929
to 1736968
and 1479251

de 2981686
, 2916946
. 2030214
la 1935072
et 1377919
le 1293964


In [ ]:
N_MOST_FREQ = 2500

In [ ]:
limited_sorted_freq_en = {k: v for i, (k, v) in enumerate(sorted_freq_en.items()) if i < N_MOST_FREQ}
limited_sorted_freq_fr = {k: v for i, (k, v) in enumerate(sorted_freq_fr.items()) if i < N_MOST_FREQ}
len(limited_sorted_freq_en), len(limited_sorted_freq_fr)  # (2500, 2500)

In [ ]:
arr = [(k, v) for i, (k, v) in enumerate(limited_sorted_freq_en.items()) if i>(N_MOST_FREQ-5)]
arr

[('fairly', 1833), ('marketing', 1831), ('ordinary', 1828), ('evident', 1827)]

## For 3 langs

In [63]:
len(input_lang.word2count), len(pivot_lang.word2count), len(output_lang.word2count)

(97899, 367223, 132962)

In [64]:
sorted_freq_en = {k: v for k, v in sorted(input_lang.word2count.items(), key=lambda item: item[1], reverse=True)}
sorted_freq_de = {k: v for k, v in sorted(pivot_lang.word2count.items(), key=lambda item: item[1], reverse=True)}
sorted_freq_fr = {k: v for k, v in sorted(output_lang.word2count.items(), key=lambda item: item[1], reverse=True)}
len(sorted_freq_en), len(sorted_freq_de), len(sorted_freq_fr)

(97899, 367223, 132962)

In [65]:
for i, (k, v) in enumerate(sorted_freq_en.items()):
  print(k, v)
  if i==5: break
print()
for i, (k, v) in enumerate(sorted_freq_de.items()):
  print(k, v)
  if i==5: break
print()
for i, (k, v) in enumerate(sorted_freq_fr.items()):
  print(k, v)
  if i==5: break

the 3468604
, 2386508
. 1777847
of 1642795
to 1527052
and 1294691

, 3221231
die 1880072
. 1782808
der 1620397
und 1268048
in 739391

de 2624144
, 2560499
. 1794556
la 1707617
et 1210702
le 1139399


In [66]:
N_MOST_FREQ = 2500

In [67]:
limited_sorted_freq_en = {k: v for i, (k, v) in enumerate(sorted_freq_en.items()) if i < N_MOST_FREQ}
limited_sorted_freq_de = {k: v for i, (k, v) in enumerate(sorted_freq_de.items()) if i < N_MOST_FREQ}
limited_sorted_freq_fr = {k: v for i, (k, v) in enumerate(sorted_freq_fr.items()) if i < N_MOST_FREQ}

In [68]:
arr = [(k, v) for i, (k, v) in enumerate(limited_sorted_freq_de.items()) if i>(N_MOST_FREQ-5)]
arr

[('besitzen', 1618),
 ('begründung', 1618),
 ('opfern', 1617),
 ('regelmäßig', 1615)]

# Filter pairs

In [69]:
filtered_idx_list = []

In [70]:
def checkValid(sent: list, dictionary):
  valid = [word in dictionary for word in sent]
  return all(valid)

## For 2 langs

In [ ]:
for i in tqdm(range(len(pairs))):
  en_sent, fr_sent = pairs[i]
  en_sent = en_sent.split('#')
  fr_sent = fr_sent.split('#')

  if checkValid(en_sent, limited_sorted_freq_en) and checkValid(fr_sent, limited_sorted_freq_fr):
    filtered_idx_list.append(i)

100%|██████████| 2051014/2051014 [00:24<00:00, 82625.04it/s]


In [ ]:
len(filtered_idx_list)

158629

In [ ]:
# N_MOST_FREQ = 50000 ---> len(filtered_idx_list) = 1908768
# N_MOST_FREQ = 10000 ---> len(filtered_idx_list) = 923161
# N_MOST_FREQ = 5000 ---> len(filtered_idx_list) = 429012
# N_MOST_FREQ = 2500 ---> len(filtered_idx_list) = 158629

## For 3 langs

In [71]:
for i in tqdm(range(len(pairs))):
  en_sent, de_sent, fr_sent = pairs[i]
  en_sent = en_sent.split('#')
  de_sent = de_sent.split('#')
  fr_sent = fr_sent.split('#')

  if checkValid(en_sent, limited_sorted_freq_en) and checkValid(de_sent, limited_sorted_freq_de) and checkValid(fr_sent, limited_sorted_freq_fr):
    filtered_idx_list.append(i)

100%|██████████| 1818194/1818194 [00:28<00:00, 64077.52it/s]


In [72]:
len(filtered_idx_list)

75002

In [ ]:
# N_MOST_FREQ = 2500 ---> len(filtered_idx_list) = 75002

# Save to new dataset

In [73]:
dataset_name = '/content/gdrive/MyDrive/Colab Notebooks/eaai24/Datasets/en-de-fr.pkl'

In [74]:
save_dataset = load_data(dataset_name)
len(save_dataset), save_dataset[2310]

load_data SUCCESS


(1818194,
 {'en': 'Resumption of the session',
  'de': 'Wiederaufnahme der Sitzungsperiode',
  'fr': 'Reprise de la session'})

In [75]:
save_pairs = []
idx_set = set(filtered_idx_list)
for i in range(len(save_dataset)):
  if i in idx_set:
    save_pairs.append(save_dataset[i])

len(save_pairs)

75002

In [76]:
with open('/content/gdrive/MyDrive/Colab Notebooks/eaai24/Datasets/endefr_75kpairs_2k5-freq-words.pkl', 'wb') as f:
  pickle.dump(save_pairs, f)